<a href="https://colab.research.google.com/github/GuillermoFidalgo/Python-for-STEM-Teachers-Workshop/blob/master/notebooks/11-ML%20Breast%20Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
path = 'https://raw.githubusercontent.com/GuillermoFidalgo/Python-for-STEM-Teachers-Workshop/master/data/wdbc.data'

df = pd.read_csv(path,header=None)

att=["radius","texture","perimeter",
              "area","smoothness","compactness","concavity",
              "concave points","symmetry","fractal dimension"]
att2 = [i+"2" for i in att]
att3 = [i+'3' for i in att]

df.columns = np.concatenate([['ID number','Diagnosis'],att,att2,att3])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.drop(columns= "ID number").iloc[:,:11].corr()

In [ ]:
plt.figure(dpi=120)
plt.matshow(df.drop(columns= "ID number").iloc[:,:11].corr(),fignum=1,cmap='coolwarm')
plt.colorbar()
plt.show()

In [ ]:
df.drop(columns = "ID number").hist(figsize=(20,20),bins=20,histtype='step')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.model_selection import cross_val_score,train_test_split


In [ ]:
X =df.drop(columns = ['ID number'] ).iloc[:,1:]
scaler =StandardScaler()
X = np.asarray(X).astype('float32')
X = scaler.fit_transform(X)


y = df.Diagnosis


In [ ]:
y [y == "M"] = 1.0
y[y =='B'] = 0.0
y = np.asarray(y).astype('float32')

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y)

In [ ]:
import keras
from keras.activations import relu
from keras.models import Sequential,save_model,Model
from keras.layers import Dense,Flatten,Input
from keras.utils.vis_utils import plot_model

In [ ]:
model = Sequential([
    Input(shape=Xtrain.shape[1]),
    Dense(32,activation= 'relu'),
    Dense(10,activation='relu'),
    Dense(1,activation='sigmoid')]
    )
model.compile(optimizer='adam', loss='mse',metrics="accuracy")

In [ ]:
model.summary()

In [ ]:
plot_model(model,show_layer_activations=True,show_shapes=True,)

In [ ]:
history = model.fit(Xtrain,ytrain,validation_split=0.2,
    epochs=50
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])

In [ ]:
model.evaluate(Xtest,ytest)

In [ ]:
ypred=model.predict(Xtest)

In [ ]:
plt.hist(ypred)

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,roc_auc_score,roc_curve,RocCurveDisplay

In [ ]:
cm=confusion_matrix(ytest,np.round(ypred.ravel()))

In [ ]:
fig=plt.figure(dpi=110)
ax=plt.subplot(111)
ConfusionMatrixDisplay(cm).plot(ax=ax,cmap='Blues')
plt.show()

In [ ]:
auc=roc_auc_score(ytest,ypred)

fpr,tpr,thresh=roc_curve(ytest,ypred)

In [ ]:
auc

In [ ]:
RocCurveDisplay(fpr=fpr,tpr=tpr,roc_auc=auc).plot();

In [ ]:
with plt.xkcd():
    plt.figure(dpi=130)
    plt.plot(fpr,tpr)
    plt.legend(['AUC \n$1 - ${:.3E}'.format(1-auc)])
    plt.ylabel('True Postive Rate')
    plt.xlabel('False Postive Rate')
    plt.show()